#### Introdução à Base de Dados do Airbnb

O conjunto de dados "Inside Airbnb", disponível no website "http://insideairbnb.com/", é uma valiosa fonte de informações sobre listagens de hospedagem, avaliações de hóspedes e disponibilidade de calendário em várias cidades ao redor do mundo, incluindo o Rio de Janeiro. Antes de prosseguirmos com a engenharia de dados, é importante entender os principais componentes deste conjunto de dados:

1. **Listing (Listagem):** Este conjunto de dados contém informações detalhadas sobre as propriedades listadas no Airbnb. Cada registro representa uma listagem individual e inclui informações como o tipo de propriedade, preço, localização, número de quartos, comodidades oferecidas e muito mais.

2. **Reviews (Avaliações):** O conjunto de dados de avaliações contém informações sobre as avaliações feitas por hóspedes que ficaram nas propriedades listadas. Ele inclui dados como a data da avaliação, o identificador da propriedade, os comentários escritos pelos hóspedes, e outras informações. 

3. **Calendar (Calendário):** Este conjunto de dados contém informações sobre a disponibilidade das propriedades ao longo do tempo. Ele lista as datas em que as propriedades estão disponíveis para reserva, bem como os preços para cada data.

O dicionário dos dados também está disponível no website: "http://insideairbnb.com/".

#### Passos do Projeto

1. **Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**
   - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
   - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

<br>

2. **Data Clean - Camada Silver:**
   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

<br>

3. **Data Quality - Camada Silver:**
   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

<br>

4. **Testes de Qualidade - Camada Silver:**
   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

<br>

5. **Transformação de Dados com dbt - Camada Silver:**
   - Utilize a ferramenta dbt para criar a camada "silver" de dados, realizando transformações e preparando os dados da camada em questão.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "silver" e automatize a execução das transformações.

<br>

6. **Armazenamento de Dados em PostgreSQL - Camada Silver:**
   - Armazene os dados da camada "silver" no mesmo banco de dados PostgreSQL.
   - Estabeleça conexões entre o dbt e o PostgreSQL para carregar os dados transformados da camada "silver" no banco.

<br>

7. **Validação de Expectativas com Great Expectations - Camada Silver:**
   - Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
   - Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

<br>

8. **Transformação de Dados com dbt - Camada Gold:**
   - Utilize o dbt para criar a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "gold" e automatize a execução das transformações.
   - Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.

<br>

 9. **Apresentação e Discussão:**
    - Apresente os resultados do projeto para a turma, enfatizando os aspectos de engenharia de dados, qualidade de dados e uso de ferramentas como dbt, Great Expectations e o armazenamento em um banco de dados PostgreSQL nas camadas "bronze", "silver" e "gold".


### 1. **Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**
    - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
    - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

In [ ]:
!pip freeze > requirements.txt

In [78]:
import os
import requests
import pandas as pd
import psycopg2 as pg
import spacy
import urllib.parse
import re
import great_expectations as gx


from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from spellchecker import SpellChecker
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# Carrega as variáveis do arquivo .env
load_dotenv()

HOST = os.getenv('HOST')
PORT = os.getenv('PORT')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')

In [8]:
# URL da página
url = "http://insideairbnb.com/get-the-data"

# Faz uma requisição à página
response = requests.get(url)

# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    # Cria um objeto BeautifulSoup para analisar o HTML
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontra todos os links na página
    links = soup.find_all("a")

    # Itera pelos links
    for link in links:
        # Obtém o o nome do arquivo
        texto_link = link.text.strip()

        # Verifica se o link contém "listings.csv.gz" e "rio-de-janeiro"
        if ("listings.csv.gz" in link or "calendar.csv.gz" in link or "reviews.csv.gz" in link) and "rio-de-janeiro" in str(link):
            # Obtém o URL completo
            link_completo = urllib.parse.urljoin(url, link['href'])

            # Faz o download do arquivo
            response = requests.get(link_completo)
            
            # Cria o file_path
            file_path = 'dados/' + texto_link            

            # Verifica se o download foi bem-sucedido
            if response.status_code == 200:
                # Salva o arquivo no disco
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                print(f"Arquivo '{texto_link}' baixado com sucesso.")
            else:
                print(f"Falha ao baixar o arquivo '{texto_link}'. Status code: {response.status_code}")
else:
    print(f"Falha ao acessar a página. Status code: {response.status_code}")


Arquivo 'listings.csv.gz' baixado com sucesso.
Arquivo 'calendar.csv.gz' baixado com sucesso.
Arquivo 'reviews.csv.gz' baixado com sucesso.


In [9]:
# Carrega o arquivo como DataFrame
df_listings = pd.read_csv(r"dados/listings.csv.gz", compression='gzip', low_memory=False)
df_reviews = pd.read_csv(r"dados/reviews.csv.gz", compression='gzip', low_memory=False)
df_calendar = pd.read_csv(r"dados/calendar.csv.gz", compression='gzip', low_memory=False)

display(df_listings.head(2))
display(df_reviews.head(2))
display(df_calendar.head(2))

In [ ]:
connection_string = f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}'

# Cria a engine
conection = create_engine(url)
print(f"Conexão: {conection}")

# Carrega os dados no banco
###try:
###    status = df_listings.to_sql(name='grupo_fantastico_listings', schema='raw', con=conection, if_exists='replace', index=False)
###    print(f'Status da operação: {status}')
###except Exception as e:
###    print(f'Ocorreu um erro: {e}')
###    
###try:
###    status = df_reviews.to_sql(name='grupo_fantastico_reviews', schema='raw', con=conection, if_exists='replace', index=False)
###    print(f'Status da operação: {status}')
###except Exception as e:
###    print(f'Ocorreu um erro: {e}')
###    
###try:
###    status = df_calendar.to_sql(name='grupo_fantastico_calendar', schema='raw', con=conection, if_exists='replace')
###    print(f'Status da operação: {status}')
###except Exception as e:
###    print(f'Ocorreu um erro: {e}')
###

### 2. **Data Clean - Camada Silver:**
   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

#### Funções para tratamento de texto

In [ ]:
"""
Formata os nomes das colunas de um DataFrame.
- Converte para minúsculas
- Substitui espaços por underscores
- Remove caracteres especiais

Args:
    df (pd.DataFrame): DataFrame com os nomes das colunas a serem formatados.
    
Returns:
    pd.DataFrame: DataFrame com os nomes de coluna formatados.
"""
def format_column_name(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('[^a-zA-Z0-9_]', '', regex=True)
    return df


# Carrega o modelo de língua portuguesa
nlp = spacy.load('pt_core_news_sm')

# Cria a instância do corretor ortográfico
spell = SpellChecker(language='pt')

"""
Limpa o texto removendo caracteres especiais e corrigindo erros de digitação.

Args:
    texto (str): O texto a ser limpo.
    
Returns:
    str: Texto limpo.
"""
def clean_text(texto):
    # Processar o texto com o spaCy
    doc = nlp(texto)
    
    # Extrair tokens e verificar se estão corretos
    tokens_corrigidos = [spell.correction(token.text) for token in doc]
    
    # Juntar os tokens corrigidos de volta em uma string
    texto_corrigido = ' '.join(tokens_corrigidos)
    
    # Remove caracteres especiais
    # bug : essa linha também remove as pontuações
    texto_limpo = re.sub(r'[^a-zA-Z0-9\s]', '', texto_corrigido)
    return texto_limpo

#### df_listings

In [26]:
print(f"df_listings.shape: {df_listings.shape}")

df_listings.shape: (31964, 75)


In [27]:
# Descarta os duplicados
df_listings = df_listings.drop_duplicates()

print(f"df_listings.shape: {df_listings.shape}")

df_listings.shape: (31964, 75)


In [85]:
df_listings[['review_scores_rating', 'review_scores_value']]

,review_scores_rating,review_scores_value
0,4.73,4.65
1,4.71,4.38
2,4.89,4.89
3,4.70,4.67
4,4.71,4.59
...,...,...
31959,NaN,NaN
31960,NaN,NaN
31961,NaN,NaN
31962,NaN,NaN


In [68]:
# Seleciona as colunas que têm nulos
columns_with_nulls = df_listings.columns[df_listings.isna().any()].tolist()
print(f"Qtd. de colunas com nulos em df_listings: {len(columns_with_nulls)}\n")

percentage_nulls = ((100 * df_listings[columns_with_nulls].isna().sum().sort_values(ascending=False)) / df_listings.shape[0]).reset_index()
percentage_nulls.columns = ['Coluna', '% de nulos']
percentage_nulls

Qtd. de colunas com nulos em df_listings: 36



,Coluna,% de nulos
0,license,100.000000
1,calendar_updated,100.000000
2,bathrooms,100.000000
3,neighbourhood_group_cleansed,100.000000
4,host_about,50.434864
5,neighbourhood,47.631711
6,neighborhood_overview,47.631711
7,review_scores_location,25.963584
8,review_scores_value,25.957327
9,review_scores_communication,25.957327


In [86]:
df_listings = df_listings.drop(columns=['host_picture_url', 'host_thumbnail_url', 'host_has_profile_pic', 'host_total_listings_count', 
'license', 'calendar_updated', 'bathrooms', 'neighbourhood_group_cleansed', 'review_scores_communication', 
'review_scores_accuracy', 'review_scores_checkin', 'review_scores_cleanliness', 'picture_url'])

In [88]:
df_listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_verifications', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       

In [89]:
df_listings.instant_bookable.unique() #Converter para boleano

array(['f', 't'], dtype=object)

In [87]:
df_listings[[]].head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,last_review,review_scores_rating,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,231497,https://www.airbnb.com/rooms/231497,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.73 · 1 bedr...,"This is a big studio at the end of Copacabana,...",NaN,https://a0.muscache.com/pictures/3582382/ee8ac...,1207700,...,2023-09-11,4.73,4.90,4.65,f,4,4,0,0,0.54
1,231516,https://www.airbnb.com/rooms/231516,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,"Special location of the building, on Copacaban...",NaN,https://a0.muscache.com/pictures/3671683/d74b4...,1207700,...,2016-08-21,4.71,4.93,4.38,f,4,4,0,0,0.20
2,236991,https://www.airbnb.com/rooms/236991,20230922043705,2023-09-23,city scrape,Rental unit in Rio de Janeiro · ★4.89 · 1 bedr...,"Aconchegante, amplo, básico, arejado, iluminad...","Copacabana, apelidada a princesinha do mar, fa...",https://a0.muscache.com/pictures/5725a59b-147d...,1241662,...,2023-09-05,4.89,4.99,4.89,f,2,2,0,0,0.65
3,17878,https://www.airbnb.com/rooms/17878,20230922043705,2023-09-23,city scrape,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,Please note that elevated rates applies for Ne...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,...,2023-09-11,4.70,4.77,4.67,f,1,1,0,0,1.87
4,25026,https://www.airbnb.com/rooms/25026,20230922043705,2023-09-22,city scrape,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,"**Fully renovated in Dec 2022, new kitchen, n...",Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/a745aa21-b8dd...,102840,...,2023-09-07,4.71,4.84,4.59,f,1,1,0,0,1.68


#### df_reviews

In [30]:
print(f"df_reviews.shape: {df_reviews.shape}")

df_reviews.shape: (637307, 6)


In [31]:
df_reviews = df_reviews.drop_duplicates()

In [32]:
print(f"df_reviews.shape: {df_reviews.shape}")

df_reviews.shape: (637307, 6)


In [63]:
# Seleciona as colunas que têm nulos
columns_with_nulls = df_reviews.columns[df_reviews.isna().any()].tolist()
print(f"Qtd. de colunas com nulos em df_reviews: {len(columns_with_nulls)}\n")

percentage_nulls = ((100 * df_reviews[columns_with_nulls].isna().sum().sort_values(ascending=False)) / df_reviews.shape[0]).reset_index()
percentage_nulls.columns = ['Coluna', '% de nulos']
percentage_nulls

Qtd. de colunas com nulos em df_reviews: 1



,Coluna,% de nulos
0,comments,0.002981


#### df_calendar

In [25]:
print(f"df_calendar.shape: {df_calendar.shape}")

df_listings.shape: (31964, 75)


In [12]:
# Descarta os duplicados
df_calendar = df_calendar.drop_duplicates()

print(f"df_calendar.shape: {df_calendar.shape}")

df_listings.shape: (31964, 75)
df_reviews.shape: (637307, 6)
df_calendar.shape: (11666976, 7)


In [62]:
# Seleciona as colunas que têm nulos
columns_with_nulls = df_calendar.columns[df_calendar.isna().any()].tolist()
print(f"Qtd. de colunas com nulos em df_listings: {len(columns_with_nulls)}\n")

percentage_nulls = ((100 * df_calendar[columns_with_nulls].isna().sum().sort_values(ascending=False)) / df_calendar.shape[0]).reset_index()
percentage_nulls.columns = ['Coluna', '% de nulos']
percentage_nulls

Qtd. de colunas com nulos em df_listings: 2



,Coluna,% de nulos
0,minimum_nights,0.000129
1,maximum_nights,0.000129


In [ ]:
# Limpeza de duplicados

### 3. **Data Quality - Camada Silver:**
   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

In [77]:
context = gx.get_context()

NameError: name 'gx' is not defined

### 4. **Testes de Qualidade - Camada Silver:**
   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.